1. 코레일 기차표 크롤링 + 예매 가능 기차표 찾기 자동화
   https://www.letskorail.com/ebizprd/prdMain.do

2. robots.txt 확인
   https://www.letskorail.com/robots.txt


In [2]:
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
# selenium 웹 드라이버 서비스 설정
service = Service(executable_path=ChromeDriverManager().install())
url = 'https://www.letskorail.com/ebizprd/EbizPrdTicketPr21111_i1.do?&txtGoAbrdDt=20240316&txtGoHour=071500&selGoYear=2024&selGoMonth=03&selGoDay=16&selGoHour=00&txtGoPage=2&txtGoStartCode=0001&txtGoStart=%EC%84%9C%EC%9A%B8&txtGoEndCode=0020&txtGoEnd=%EB%B6%80%EC%82%B0&selGoTrain=05&selGoRoom=&selGoRoom1=&txtGoTrnNo=&useSeatFlg=&useServiceFlg=&selGoSeat=&selGoService=&txtPnrNo=&hidRsvChgNo=&hidStlFlg=&radJobId=1&SeandYo=&hidRsvTpCd=03&selGoSeat1=015&selGoSeat2=&txtPsgCnt1=1&txtPsgCnt2=0&txtMenuId=11&txtPsgFlg_1=1&txtPsgFlg_2=0&txtPsgFlg_3=0&txtPsgFlg_4=0&txtPsgFlg_5=0&txtPsgFlg_8=0&chkCpn=N&txtSeatAttCd_4=015&txtSeatAttCd_3=000&txtSeatAttCd_2=000&txtGoStartCode2=&txtGoEndCode2=&hidDiscount=&hidEasyTalk=&adjcCheckYn=N'

driver = webdriver.Chrome(service=service)
driver.get(url)
wait = WebDriverWait(driver, 10) #실행 X > 최대 10초간 대기

# selenium으로 자바 스크립트 실행된 후의 페이지 소스를 가져옴
html = driver.page_source
driver.quit()

# BeautifulSoup으로 파싱
soup = BeautifulSoup(html, 'html.parser')


In [4]:
table_contents = soup.find('tbody')
table_contents

<tbody><tr class="">
<td>
								직통
								</td>
<td class="bdl_on bg-02" title="무궁화호">
<a href="javascript:btnTimeInfo('/ebizprd/EbizPrdTicketPr11131_i1.do?txtRunDt=20240316&amp;txtDptDt=20240316&amp;txtTrnNo=01205&amp;txtTrnGpCd=102');" title="열차운행 시각조회 새창 열기">
<span class="point-02">
									1205
									</span>
</a>
</td>
<td>
							서울<br/>07:15
							</td>
<td>
							부산<br/>12:58
							</td>
<td>
							-
							</td>
<td>
<img alt="좌석매진" src="/docs/2007/img/common/btn_selloff.gif"/>
</td>
<td>
								-
								</td>
<td>
<img alt="역발매중" src="/docs/2007/img/common/btn_seet_rsv2.gif"/>
</td>
<!-- 365할인 -->
<td class="guide365">
<div class="shwSchPrice1"><strong>28,600원</strong></div>
</td>
<!--/////////////////////////////////////////////////////////////////////// -->
<!-- 예약대기 예약가능 여부  표시                                           -->
<!-- 직통이면서 단일속성(일반) 요구시에만 표시                              -->
<!--/////////////////////////////////////////////////////////////////////// --

In [5]:
#빈 리스트 만들기
data_rows = []

#table_contents의 각 tr태그를 반복 > td 데이터를 컬럼에 담기
for tr in table_contents.find_all('tr'):
    #각 열에 해당하는 데이터 추출
    data = []
    for td in tr.find_all('td'):
        text = td.get_text(strip=True)
        #열에 데이터를 추가
        data.append(text)
        
    #행에 데이터를 추가
    data_rows.append(data)
    
#데이터 프레임 생성
#columns 이름
columns = [
    '구분', '열차번호', '출발시각', '도착시각', '특실/우등실', '일반실', '유아',
    '자유석/입석', '인터넷특가(멤버십혜택)', '예약대기', '정차역(경유)', '차량유형/편성정보', '운임요금', '소요시간'
]

df = pd.DataFrame(data_rows, columns=columns)
df

,구분,열차번호,출발시각,도착시각,특실/우등실,일반실,유아,자유석/입석,인터넷특가(멤버십혜택),예약대기,정차역(경유),차량유형/편성정보,운임요금,소요시간
0,직통,1205,서울07:15,부산12:58,-,,-,,"28,600원",,-,,,05:43
1,직통,KTX011,서울07:27,부산10:09,,,,,"59,800원(5% 적립)",,-,,,02:42
2,직통,KTX013,서울07:49,부산10:19,,,,,"59,800원(5% 적립)",,-,,,02:30
3,직통,KTX015,서울07:57,부산10:45,,,,,"59,800원(5% 적립)",,-,,,02:48
4,직통,KTX121,서울08:11,부산11:32,,,,,"48,800원(5% 적립)",,,,,03:21
5,직통,KTX191,서울08:34,부산11:03,,,,,"59,800원(5% 적립)",,-,,,02:29
6,직통,1003,서울08:48,부산13:42,-,,,,"42,600원",,-,,,04:54
7,직통,KTX017,서울08:57,부산11:15,,,,,"59,800원(5% 적립)",,-,,,02:18
8,직통,KTX019,서울09:27,부산12:16,,,,,"59,800원(5% 적립)",,-,,,02:49
9,직통,KTX101,서울09:32,부산12:52,,,,,"53,500원(5% 적립)",,,,,03:20


In [6]:
#1. 이미지 코드에 대응

#빈 리스트 만들기
data_rows = []

#table_contents의 각 tr태그를 반복 > td 데이터를 컬럼에 담기
for tr in table_contents.find_all('tr'):
    #각 열에 해당하는 데이터 추출
    data = []
    
    for td in tr.find_all('td'):
        #td 안에 img 태그가 있으면 alt 속성 추출
        img_tag = td.find('img')
        
        #img_tag가 존재하는지 확인
        if img_tag: #img_tag O
            text = img_tag.get('alt','')
        else: #img_tag X
            text = td.get_text(strip=True)
        
        #열에 데이터를 추가
        data.append(text)
        
    #행에 데이터를 추가
    data_rows.append(data)
    
#데이터 프레임 생성
#columns 이름
columns = [
    '구분', '열차번호', '출발시각', '도착시각', '특실/우등실', '일반실', '유아',
    '자유석/입석', '인터넷특가(멤버십혜택)', '예약대기', '정차역(경유)', '차량유형/편성정보', '운임요금', '소요시간'
]

df = pd.DataFrame(data_rows, columns=columns)
df

,구분,열차번호,출발시각,도착시각,특실/우등실,일반실,유아,자유석/입석,인터넷특가(멤버십혜택),예약대기,정차역(경유),차량유형/편성정보,운임요금,소요시간
0,직통,1205,서울07:15,부산12:58,-,좌석매진,-,역발매중,"28,600원",매진,-,전동휠체어,조회,05:43
1,직통,KTX011,서울07:27,부산10:09,좌석매진,좌석매진,유아동반객실,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:42
2,직통,KTX013,서울07:49,부산10:19,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:30
3,직통,KTX015,서울07:57,부산10:45,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:48
4,직통,KTX121,서울08:11,부산11:32,좌석매진,좌석매진,좌석매진,좌석매진,"48,800원(5% 적립)",매진,경부기존선 수원역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다,전동휠체어,조회,03:21
5,직통,KTX191,서울08:34,부산11:03,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,KTX-산천 AType,조회,02:29
6,직통,1003,서울08:48,부산13:42,-,좌석매진,유아동반객실,역발매중,"42,600원",매진,-,전동휠체어,조회,04:54
7,직통,KTX017,서울08:57,부산11:15,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:18
8,직통,KTX019,서울09:27,부산12:16,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,KTX-산천 AType,조회,02:49
9,직통,KTX101,서울09:32,부산12:52,좌석매진,좌석매진,유아동반객실,좌석매진,"53,500원(5% 적립)",매진,"경부기존선 서대구, 구포역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다",전동휠체어,조회,03:20


In [7]:
#2. url 포함된 데이터 프레임 생성

# selenium 웹 드라이버 서비스 설정
service = Service(executable_path=ChromeDriverManager().install())
url = 'https://www.letskorail.com/ebizprd/EbizPrdTicketPr21111_i1.do?&txtGoAbrdDt=20240316&txtGoHour=071500&selGoYear=2024&selGoMonth=03&selGoDay=16&selGoHour=00&txtGoPage=2&txtGoStartCode=0001&txtGoStart=%EC%84%9C%EC%9A%B8&txtGoEndCode=0020&txtGoEnd=%EB%B6%80%EC%82%B0&selGoTrain=05&selGoRoom=&selGoRoom1=&txtGoTrnNo=&useSeatFlg=&useServiceFlg=&selGoSeat=&selGoService=&txtPnrNo=&hidRsvChgNo=&hidStlFlg=&radJobId=1&SeandYo=&hidRsvTpCd=03&selGoSeat1=015&selGoSeat2=&txtPsgCnt1=1&txtPsgCnt2=0&txtMenuId=11&txtPsgFlg_1=1&txtPsgFlg_2=0&txtPsgFlg_3=0&txtPsgFlg_4=0&txtPsgFlg_5=0&txtPsgFlg_8=0&chkCpn=N&txtSeatAttCd_4=015&txtSeatAttCd_3=000&txtSeatAttCd_2=000&txtGoStartCode2=&txtGoEndCode2=&hidDiscount=&hidEasyTalk=&adjcCheckYn=N'

driver = webdriver.Chrome(service=service)
driver.get(url)
wait = WebDriverWait(driver, 10) #실행 X > 최대 10초간 대기

# selenium으로 자바 스크립트 실행된 후의 페이지 소스를 가져옴
html = driver.page_source
driver.quit()

# BeautifulSoup으로 파싱
soup = BeautifulSoup(html, 'html.parser')

#table_contents로 필요한 부분만 정의
table_contents = soup.find('tbody')

#빈 리스트 만들기
data_rows = []

#table_contents의 각 tr태그를 반복 > td 데이터를 컬럼에 담기
for tr in table_contents.find_all('tr'):
    #각 열에 해당하는 데이터 추출
    data = []
    
    for td in tr.find_all('td'):
        #td 안에 img 태그가 있으면 alt 속성 추출
        img_tag = td.find('img')
        
        #img_tag가 존재하는지 확인
        if img_tag: #img_tag O
            text = img_tag.get('alt','')
        else: #img_tag X
            text = td.get_text(strip=True)
        
        #열에 데이터를 추가
        data.append(text)
    #data에 url 추가
    data.append(url)
        
    #행에 데이터를 추가
    data_rows.append(data)
    
#데이터 프레임 생성
#columns 이름
columns = [
    '구분', '열차번호', '출발시각', '도착시각', '특실/우등실', '일반실', '유아',
    '자유석/입석', '인터넷특가(멤버십혜택)', '예약대기', '정차역(경유)', '차량유형/편성정보', '운임요금', '소요시간', 'URL'
]

df = pd.DataFrame(data_rows, columns=columns)
df

,구분,열차번호,출발시각,도착시각,특실/우등실,일반실,유아,자유석/입석,인터넷특가(멤버십혜택),예약대기,정차역(경유),차량유형/편성정보,운임요금,소요시간,URL
0,직통,1205,서울07:15,부산12:58,-,좌석매진,-,역발매중,"28,600원",매진,-,전동휠체어,조회,05:43,https://www.letskorail.com/ebizprd/EbizPrdTick...
1,직통,KTX011,서울07:27,부산10:09,좌석매진,좌석매진,유아동반객실,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:42,https://www.letskorail.com/ebizprd/EbizPrdTick...
2,직통,KTX013,서울07:49,부산10:19,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:30,https://www.letskorail.com/ebizprd/EbizPrdTick...
3,직통,KTX015,서울07:57,부산10:45,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:48,https://www.letskorail.com/ebizprd/EbizPrdTick...
4,직통,KTX121,서울08:11,부산11:32,좌석매진,좌석매진,좌석매진,좌석매진,"48,800원(5% 적립)",매진,경부기존선 수원역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다,전동휠체어,조회,03:21,https://www.letskorail.com/ebizprd/EbizPrdTick...
5,직통,KTX191,서울08:34,부산11:03,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,KTX-산천 AType,조회,02:29,https://www.letskorail.com/ebizprd/EbizPrdTick...
6,직통,1003,서울08:48,부산13:42,-,좌석매진,유아동반객실,역발매중,"42,600원",매진,-,전동휠체어,조회,04:54,https://www.letskorail.com/ebizprd/EbizPrdTick...
7,직통,KTX017,서울08:57,부산11:15,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:18,https://www.letskorail.com/ebizprd/EbizPrdTick...
8,직통,KTX019,서울09:27,부산12:16,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,KTX-산천 AType,조회,02:49,https://www.letskorail.com/ebizprd/EbizPrdTick...
9,직통,KTX101,서울09:32,부산12:52,좌석매진,좌석매진,좌석매진,좌석매진,"53,500원(5% 적립)",매진,"경부기존선 서대구, 구포역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다",전동휠체어,조회,03:20,https://www.letskorail.com/ebizprd/EbizPrdTick...


In [12]:
#3. Xpath 사용, 다음 버튼 클릭, URL 가져오기
from selenium.webdriver.common.by import By
import time

# selenium 웹 드라이버 서비스 설정
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

url = 'https://www.letskorail.com/ebizprd/EbizPrdTicketPr21111_i1.do?&txtGoAbrdDt=20240316&txtGoHour=071500&selGoYear=2024&selGoMonth=03&selGoDay=16&selGoHour=00&txtGoPage=2&txtGoStartCode=0001&txtGoStart=%EC%84%9C%EC%9A%B8&txtGoEndCode=0020&txtGoEnd=%EB%B6%80%EC%82%B0&selGoTrain=05&selGoRoom=&selGoRoom1=&txtGoTrnNo=&useSeatFlg=&useServiceFlg=&selGoSeat=&selGoService=&txtPnrNo=&hidRsvChgNo=&hidStlFlg=&radJobId=1&SeandYo=&hidRsvTpCd=03&selGoSeat1=015&selGoSeat2=&txtPsgCnt1=1&txtPsgCnt2=0&txtMenuId=11&txtPsgFlg_1=1&txtPsgFlg_2=0&txtPsgFlg_3=0&txtPsgFlg_4=0&txtPsgFlg_5=0&txtPsgFlg_8=0&chkCpn=N&txtSeatAttCd_4=015&txtSeatAttCd_3=000&txtSeatAttCd_2=000&txtGoStartCode2=&txtGoEndCode2=&hidDiscount=&hidEasyTalk=&adjcCheckYn=N'

#빈 리스트 만들기
data_rows = []

#3번 반복
for _ in range(3):
    driver.get(url) #갱신된 url 반복적으로 생성
    time.sleep(2) #2초 대기

    # selenium으로 자바 스크립트 실행된 후의 페이지 소스를 가져옴
    html = driver.page_source

    # BeautifulSoup으로 파싱
    soup = BeautifulSoup(html, 'html.parser')

    #table_contents로 필요한 부분만 정의
    table_contents = soup.find('tbody')
    
    #table_contents의 각 tr태그를 반복 > td 데이터를 컬럼에 담기
    for tr in table_contents.find_all('tr'):
        #각 열에 해당하는 데이터 추출
        data = []
        
        for td in tr.find_all('td'):
            #td 안에 img 태그가 있으면 alt 속성 추출
            img_tag = td.find('img')
            
            #img_tag가 존재하는지 확인
            if img_tag: #img_tag O
                text = img_tag.get('alt','')
            else: #img_tag X
                text = td.get_text(strip=True)
        
            #열에 데이터를 추가
            data.append(text)
        #data에 url 추가
        data.append(url)
        #행에 데이터를 추가
        data_rows.append(data)

    #'다음' 버튼 찾아서 클릭
    try:
        next_button = driver.find_element(By.XPATH, '//*[@id="divResult"]/table[2]/tbody/tr/td/a[2]') #copy XPATH
    except: #이전 버튼이 없으면 '다음; 버튼 위치가 1번째 a태그로 변경되는거 대응 
        next_button = driver.find_element(By.XPATH, '//*[@id="divResult"]/table[2]/tbody/tr/td/a[1]') #copy XPATH
    
    next_button.click()
    time.sleep(3)
    
    #'다음' 버튼 누르고 이동한 페이지의 URL을 다시 url 변수에 대입
    url = driver.current_url

driver.quit()

#데이터 프레임 생성
#columns 이름
columns = [
    '구분', '열차번호', '출발시각', '도착시각', '특실/우등실', '일반실', '유아',
    '자유석/입석', '인터넷특가(멤버십혜택)', '예약대기', '정차역(경유)', '차량유형/편성정보', '운임요금', '소요시간', 'URL'
]

df = pd.DataFrame(data_rows, columns=columns)
df


,구분,열차번호,출발시각,도착시각,특실/우등실,일반실,유아,자유석/입석,인터넷특가(멤버십혜택),예약대기,정차역(경유),차량유형/편성정보,운임요금,소요시간,URL
0,직통,1205,서울07:15,부산12:58,-,좌석매진,-,역발매중,"28,600원",매진,-,전동휠체어,조회,05:43,https://www.letskorail.com/ebizprd/EbizPrdTick...
1,직통,KTX011,서울07:27,부산10:09,좌석매진,좌석매진,유아동반객실,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:42,https://www.letskorail.com/ebizprd/EbizPrdTick...
2,직통,KTX013,서울07:49,부산10:19,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:30,https://www.letskorail.com/ebizprd/EbizPrdTick...
3,직통,KTX015,서울07:57,부산10:45,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:48,https://www.letskorail.com/ebizprd/EbizPrdTick...
4,직통,KTX121,서울08:11,부산11:32,좌석매진,좌석매진,좌석매진,좌석매진,"48,800원(5% 적립)",매진,경부기존선 수원역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다,전동휠체어,조회,03:21,https://www.letskorail.com/ebizprd/EbizPrdTick...
5,직통,KTX191,서울08:34,부산11:03,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,KTX-산천 AType,조회,02:29,https://www.letskorail.com/ebizprd/EbizPrdTick...
6,직통,1003,서울08:48,부산13:42,-,좌석매진,유아동반객실,역발매중,"42,600원",매진,-,전동휠체어,조회,04:54,https://www.letskorail.com/ebizprd/EbizPrdTick...
7,직통,KTX017,서울08:57,부산11:15,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:18,https://www.letskorail.com/ebizprd/EbizPrdTick...
8,직통,KTX019,서울09:27,부산12:16,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,KTX-산천 AType,조회,02:49,https://www.letskorail.com/ebizprd/EbizPrdTick...
9,직통,KTX101,서울09:32,부산12:52,좌석매진,좌석매진,좌석매진,좌석매진,"53,500원(5% 적립)",매진,"경부기존선 서대구, 구포역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다",전동휠체어,조회,03:20,https://www.letskorail.com/ebizprd/EbizPrdTick...


In [14]:
df = df.drop_duplicates(subset=['출발시각'], keep='first')
df.head(20)

,구분,열차번호,출발시각,도착시각,특실/우등실,일반실,유아,자유석/입석,인터넷특가(멤버십혜택),예약대기,정차역(경유),차량유형/편성정보,운임요금,소요시간,URL
0,직통,1205,서울07:15,부산12:58,-,좌석매진,-,역발매중,"28,600원",매진,-,전동휠체어,조회,05:43,https://www.letskorail.com/ebizprd/EbizPrdTick...
1,직통,KTX011,서울07:27,부산10:09,좌석매진,좌석매진,유아동반객실,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:42,https://www.letskorail.com/ebizprd/EbizPrdTick...
2,직통,KTX013,서울07:49,부산10:19,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:30,https://www.letskorail.com/ebizprd/EbizPrdTick...
3,직통,KTX015,서울07:57,부산10:45,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:48,https://www.letskorail.com/ebizprd/EbizPrdTick...
4,직통,KTX121,서울08:11,부산11:32,좌석매진,좌석매진,좌석매진,좌석매진,"48,800원(5% 적립)",매진,경부기존선 수원역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다,전동휠체어,조회,03:21,https://www.letskorail.com/ebizprd/EbizPrdTick...
5,직통,KTX191,서울08:34,부산11:03,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,KTX-산천 AType,조회,02:29,https://www.letskorail.com/ebizprd/EbizPrdTick...
6,직통,1003,서울08:48,부산13:42,-,좌석매진,유아동반객실,역발매중,"42,600원",매진,-,전동휠체어,조회,04:54,https://www.letskorail.com/ebizprd/EbizPrdTick...
7,직통,KTX017,서울08:57,부산11:15,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,전동휠체어,조회,02:18,https://www.letskorail.com/ebizprd/EbizPrdTick...
8,직통,KTX019,서울09:27,부산12:16,좌석매진,좌석매진,좌석매진,역발매중,"59,800원(5% 적립)",매진,-,KTX-산천 AType,조회,02:49,https://www.letskorail.com/ebizprd/EbizPrdTick...
9,직통,KTX101,서울09:32,부산12:52,좌석매진,좌석매진,좌석매진,좌석매진,"53,500원(5% 적립)",매진,"경부기존선 서대구, 구포역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다",전동휠체어,조회,03:20,https://www.letskorail.com/ebizprd/EbizPrdTick...


In [18]:
fast_reserve_df = df[df['일반실']=='예약하기']
#인덱스 리셋
fast_reserve_df = fast_reserve_df.reset_index(drop=True)
fast_reserve_df.head()

,구분,열차번호,출발시각,도착시각,특실/우등실,일반실,유아,자유석/입석,인터넷특가(멤버십혜택),예약대기,정차역(경유),차량유형/편성정보,운임요금,소요시간,URL
0,직통,4201,서울11:49,부산16:07,-,예약하기,유아동반객실,역발매중,"42,600원",-,-,ITX-마음,조회,04:18,https://www.letskorail.com/ebizprd/EbizPrdTick...
1,직통,1209,서울13:03,부산18:59,-,예약하기,-,역발매중,"28,600원",-,-,,조회,05:56,https://www.letskorail.com/ebizprd/EbizPrdTick...
2,직통,KTX103,서울13:07,부산16:27,예약하기,예약하기,유아동반객실,역발매중,"53,500원(5% 적립)",-,"경부기존선 서대구, 구포역을 정차하는 KTX 입니다\n소요시간과 운임의 차이가 있습니다",전동휠체어,조회,03:20,https://www.letskorail.com/ebizprd/EbizPrdTick...
3,직통,KTX033,서울13:17,부산15:58,예약하기,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:41,https://www.letskorail.com/ebizprd/EbizPrdTick...
4,직통,KTX083,서울13:48,부산16:24,예약하기,예약하기,유아동반객실,역발매중,"59,800원(5% 적립)",-,-,전동휠체어,조회,02:36,https://www.letskorail.com/ebizprd/EbizPrdTick...


In [20]:
fast_reserve_df.loc[0, 'URL']

'https://www.letskorail.com/ebizprd/EbizPrdTicketPr21111_i1.do?&txtGoAbrdDt=20240316&txtGoHour=113000&selGoYear=2024&selGoMonth=03&selGoDay=16&selGoHour=00&txtGoPage=2&txtGoStartCode=0001&txtGoStart=%EC%84%9C%EC%9A%B8&txtGoEndCode=0020&txtGoEnd=%EB%B6%80%EC%82%B0&selGoTrain=05&selGoRoom=&selGoRoom1=&txtGoTrnNo=&useSeatFlg=&useServiceFlg=&selGoSeat=&selGoService=&txtPnrNo=&hidRsvChgNo=&hidStlFlg=&radJobId=1&SeandYo=&hidRsvTpCd=03&selGoSeat1=015&selGoSeat2=&txtPsgCnt1=1&txtPsgCnt2=0&txtMenuId=11&txtPsgFlg_1=1&txtPsgFlg_2=0&txtPsgFlg_3=0&txtPsgFlg_4=0&txtPsgFlg_5=0&txtPsgFlg_8=0&chkCpn=N&txtSeatAttCd_4=015&txtSeatAttCd_3=000&txtSeatAttCd_2=000&txtGoStartCode2=&txtGoEndCode2=&hidDiscount=&hidEasyTalk=&adjcCheckYn=N'

In [ ]:
fast_reserve_df.to_csv('korail_fast_reserve.csv', index=False)